# Chapter 49: Knowledge Graph Integration with Neo4j
Cypher simulation, entity extraction, graph-based RAG.


In [ ]:
import json, re
print("Knowledge Graph ready")


In [ ]:
class GraphDB:
    def __init__(self): self.nodes={}; self.edges=[]; self._id=0
    def node(self,label,**props): nid=self._id; self._id+=1; self.nodes[nid]={"id":nid,"label":label,**props}; return nid
    def edge(self,src,dst,rel): self.edges.append({"src":src,"dst":dst,"rel":rel})
    def match(self,label=None,**filt):
        r=list(self.nodes.values())
        if label: r=[n for n in r if n.get("label")==label]
        for k,v in filt.items(): r=[n for n in r if n.get(k)==v]
        return r
db=GraphDB()
h1=db.node("Host",ip="10.0.1.10",hostname="web-srv-1")
h2=db.node("Host",ip="10.0.1.11",hostname="db-srv-1")
fw=db.node("Firewall",name="FW-Core",vendor="Palo Alto")
db.edge(h1,fw,"TRAVERSES"); db.edge(fw,h2,"ALLOWS")
print("Hosts:",[n["hostname"] for n in db.match("Host")])
print("Edges:",[(e["rel"],e["src"],"->",e["dst"]) for e in db.edges])


In [ ]:
def extract(text): return {"ips":re.findall(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",text),"ports":re.findall(r"port\s+(\d+)",text,re.I)}
logs=["Connection from 192.168.1.5 to 10.0.0.1 port 22 blocked","CVE-2024-5678 on 172.16.0.10 port 443","SSH from 203.0.113.42 port 22"]
for log in logs: e=extract(log); print(f"Log: {log[:45]} => {e}")
def graph_rag(db,ips):
    ctx=[]; rel=[]
    for ip in ips:
        nodes=db.match(ip=ip); ctx.extend(nodes)
        for n in nodes:
            for e in db.edges:
                oid=e["dst"] if e["src"]==n["id"] else (e["src"] if e["dst"]==n["id"] else None)
                if oid is not None and oid in db.nodes: rel.append({"rel":e["rel"],"node":db.nodes[oid]})
    return ctx,rel
ctx,rel=graph_rag(db,["10.0.1.10"])
print(f"Context: {len(ctx)} nodes")
for r in rel: print(f"  -[{r[chr(114)+chr(101)+chr(108)]}]-> {r[chr(110)+chr(111)+chr(100)+chr(101)]}")
